In [1]:
import torch
import numpy as np
import pytorch_lightning as pl
import torchvision.transforms as tf
from tqdm import tqdm
from predict import *
from HGGEP_HGNN import *
from dataset import ViT_HER2ST, ViT_SKIN
from scipy.stats import pearsonr,spearmanr
from torch.utils.data import DataLoader
from pytorch_lightning.loggers import TensorBoardLogger
from copy import deepcopy as dcp
from collections import defaultdict as dfd
from sklearn.metrics import adjusted_rand_score as ari_score
from sklearn.metrics.cluster import normalized_mutual_info_score as nmi_score

/home/zy/anaconda3/envs/gtn/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/zy/anaconda3/envs/gtn/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[easydl] tensorflow not available!


# Data Loading

In [2]:
name=[*[f'A{i}' for i in range(2,7)],*[f'B{i}' for i in range(1,7)],
      *[f'C{i}' for i in range(1,7)],*[f'D{i}' for i in range(1,7)],
      *[f'E{i}' for i in range(1,4)],*[f'F{i}' for i in range(1,4)],*[f'G{i}' for i in range(1,4)]]
patients = ['P2', 'P5', 'P9', 'P10']
reps = ['rep1', 'rep2', 'rep3']
skinname = []
for i in patients:
    for j in reps:
        skinname.append(i+'_ST_'+j)
device='cuda'
tag='5-7-2-8-4-16-32'
k,p,d1,d2,d3,h,c=map(lambda x:int(x),tag.split('-'))
dropout=0.2
random.seed(12000)
np.random.seed(12000)
torch.manual_seed(12000)
torch.cuda.manual_seed(12000)
torch.cuda.manual_seed_all(12000)  
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
fold=12
data='her2st'
prune='Grid' if data=='her2st' else 'NA'
genes=171 if data=='cscc' else 785
trainset = pk_load(fold,'train',dataset=data,flatten=False,adj=True,ori=True,prune=prune)
train_loader = DataLoader(trainset, batch_size=1, num_workers=0, shuffle=True)

te_names: ['C2']
Loading imgs...
Loading metadata...


# HGGEP training

In [4]:
model=HGGEP(
    depth1=d1, depth2=d2,depth3=d3,n_genes=genes,
    kernel_size=k, patch_size=p,
    heads=h, channel=c, dropout=0.2,
    zinb=0.25, nb=False,
    bake=5, lamb=0.5, 
)

In [ ]:
logger=None
trainer = pl.Trainer(
    gpus=[0], max_epochs=350,
    logger=logger,
)
trainer.fit(model, train_loader)

import os
if not os.path.isdir("./model/"):
    os.mkdir("./model/")

torch.save(model.state_dict(),f"./model/her2st/{fold}-HGGEP.ckpt")

# HGGEP Prediction

In [7]:
fold=12
data='her2st'
prune='Grid' if data=='her2st' else 'NA'
genes=171 if data=='cscc' else 785

In [13]:
testset = pk_load(fold,'test',dataset=data,flatten=False,adj=True,ori=True,prune=prune)
test_loader = DataLoader(testset, batch_size=1, num_workers=0, shuffle=False)
label=testset.label[testset.names[0]]
genes=785
model=HGGEP(
    depth1=d1, depth2=d2,depth3=d3,n_genes=genes, 
    kernel_size=k, patch_size=p,
    heads=h, channel=c, dropout=0.2,
    zinb=0.25, nb=False,
    bake=5, lamb=0.5,
)
model.load_state_dict(torch.load(f'./model/her2st/{fold}-HGGEP.ckpt'))
pred, gt = test(model, test_loader,'cuda')

te_names: ['C2']
Loading imgs...
Loading metadata...
{'C2': array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.96906871, 0.        , 0.96906871, ..., 0.        , 0.96906871,
        1.24613169],
       ...,
       [0.        , 0.        , 0.        , ..., 1.12041341, 1.12041341,
        0.        ],
       [0.        , 1.62365932, 0.        , ..., 0.        , 1.16672282,
        1.16672282],
       [0.        , 1.11266272, 0.        , ..., 0.        , 0.        ,
        1.11266272]])}


100%|██████████| 1/1 [00:01<00:00,  1.40s/it]


In [14]:
R=get_R(pred,gt)[0]
print('Pearson Correlation:',np.nanmedian(R))

Pearson Correlation: 0.18960114294482458
